In [1]:
import glob, re
import numpy as np
import pandas as pd
from sklearn import *
import datetime as dt
import gc
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb

C:\Users\vedpk\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\vedpk\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\vedpk\Anaconda3\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [2]:
train = pd.read_csv("../input/train_proc5.csv")
test = pd.read_csv("../input/test_proc5.csv")
submission = pd.read_csv("../input/sample_submission.csv")
air_store = pd.read_csv("../input/allstore_info_proc1.csv").rename(columns={'store_id':'air_store_id'})
date_info = pd.read_csv("../input/date_info.csv").rename(columns={"calendar_date" : 'visit_date'})
train_weather = pd.read_csv("../input/train_weather_01.csv")
test_weather = pd.read_csv("../input/test_weather_01.csv")

In [3]:
submission['visit_date'] = submission['id'].map(lambda x: str(x).split('_')[2])
submission['air_store_id'] = submission['id'].map(lambda x: '_'.join(x.split('_')[:2]))
submission['visit_date'] = pd.to_datetime(submission['visit_date'])
submission['visitors'] = np.nan

In [4]:
all_weather = pd.concat([train_weather, test_weather])
all_weather['visit_date'] = pd.to_datetime(all_weather['visit_date'])

In [5]:
train['visit_date'] = pd.to_datetime(train['visit_date'])
test['visit_date'] = pd.to_datetime(test['visit_date'])
#test['visitors'] = -1
test['visitors'] = np.nan

In [6]:
date_info['visit_date'] = pd.to_datetime(date_info['visit_date'])

In [7]:
date_info['date_seq_id'] = date_info['visit_date'].dt.date - date_info['visit_date'].dt.date.min() + dt.timedelta(1)
date_info['date_seq_id'] = date_info['date_seq_id']/dt.timedelta(1)

In [8]:
date_info['week_seq_id'] = ((date_info['date_seq_id']+4)/7).astype(np.int64)

In [9]:
stores_to_drop = list(set(train['air_store_id']) - set(test['air_store_id']))
train = train.where(~(train['air_store_id'].isin(stores_to_drop)))
train = train.dropna(axis=0,subset=['air_store_id'])

In [10]:
all_data = pd.concat([train,test])

In [11]:
train_len = len(train)

In [12]:
del train; del test;
gc.collect();

In [13]:
all_data = pd.merge(all_data,air_store,how='left',on='air_store_id',)

In [14]:
all_data = pd.merge(all_data,date_info,how='left',on='visit_date')

In [15]:
all_data['visit_month'] = all_data['visit_date'].dt.month
all_data['visit_year'] = all_data['visit_date'].dt.year

In [16]:
all_data['diff_max_lat'] = all_data['latitude'].max() - all_data['latitude']
all_data['diff_min_lat'] = all_data['latitude'].min() - all_data['latitude']
all_data['diff_max_long'] = all_data['longitude'].max() - all_data['longitude']
all_data['diff_min_long'] = all_data['longitude'].min() - all_data['longitude']
all_data['lat_plus_long'] = all_data['latitude'] + all_data['longitude']

In [17]:
stores_to_drop = ['air_b2d8bc9c88b85f96',
 'air_cf22e368c1a71d53',
 'air_229d7e508d9f1b5e',
 'air_d0a7bd3339c3d12a',
 'air_cb083b4789a8d3a2',
 'air_2703dcb33192b181',
 'air_0ead98dd07e7a82a',
 'air_d63cfa6d6ab78446']

In [18]:
all_data = all_data.where(~(all_data['air_store_id'].isin(stores_to_drop)))
all_data = all_data.dropna(axis=0,subset=['air_store_id'])

In [19]:
all_data = pd.merge(all_data,all_weather, on=['air_store_id','visit_date'], how='left')

In [20]:
all_data.shape

(282487, 73)

In [21]:
del air_store;del date_info;gc.collect();

In [22]:
all_data['visit_date'] = pd.to_datetime(all_data['visit_date'])

In [23]:
all_data['visit_date'] = all_data['visit_date'].dt.date

In [24]:
#train.sort_values('visit_date').iloc[-1]['visit_date']-dt.timedelta(days=90) #38days validation set
print(all_data['visit_date'].min())
print(all_data['visit_date'].max())

2016-01-01
2017-05-31


In [25]:
#all_data = all_data.sort_values(['air_store_id','visit_date'])

In [26]:
train = all_data[:train_len]
test = all_data[train_len:]

In [27]:
train.shape

(250468, 73)

In [28]:
train = train.reset_index().drop('index',axis=1)
test = test.reset_index().drop('index',axis=1)

In [29]:
test.shape

(32019, 73)

In [30]:
dt.date(2017, 4, 22) + dt.timedelta(days=39)

datetime.date(2017, 5, 31)

In [31]:
train[train['visitors'] == 1].groupby('air_store_id').count()['visitors'].sort_values()[550:]

air_store_id
air_7bacc4d36fb094c9     23
air_457efe8c3a30ea17     23
air_256be208a979e023     24
air_c26f027b5bc1f081     25
air_901925b628677c2e     25
air_3a8a3f8fb5cd7f88     25
air_c8265ecc116f2284     25
air_35c4732dcbfe31be     25
air_646b93e336f0dded     25
air_1e665503b8474c55     26
air_fff68b929994bfbd     26
air_9f277fb7a2c1d842     26
air_9e920b758503ef54     27
air_2a184c1745274b2b     29
air_9a6f6e7f623003d2     29
air_382f5ace4e2247b8     30
air_22682e965418936f     30
air_9d3482b4680aee88     30
air_bed603c423b7d9d4     30
air_fdcfef8bd859f650     30
air_eb20a89bba7dd3d0     31
air_931a8a4321b6e7d1     31
air_f1094dbf2aef85d9     31
air_f85e21e543cf44f2     31
air_9dd7d38b0f1760c4     32
air_fa4ffc9057812fa2     32
air_70e9e8cd55879414     32
air_4c665a2bfff0da3b     32
air_d54d6fcb116fbed3     32
air_caf996ac27206301     34
air_aed3a8b49abe4a48     34
air_93dd7070c9bf5453     34
air_c9f6de13be8b8f25     35
air_b8925441167c3152     37
air_57013002b912772b     39
air_c75

In [32]:
all_data['']
cols_to_encode = ([i for i,j in zip(all_data.dtypes.index,all_data.dtypes.values) if j == 'object'])

lbl = preprocessing.LabelEncoder()
for i in cols_to_encode:
    all_data[i] = lbl.fit_transform(all_data[i])

In [33]:
cols_to_encode

['air_store_id',
 'visit_date',
 'genre_name',
 'area_name',
 'prefecture',
 'city',
 'day_of_week',
 'station_id']

In [34]:
def calc_shifted_ewm(series, alpha, adjust=True, days=0): #
    return series.shift(periods=days).ewm(alpha=alpha, adjust=adjust).mean()

In [35]:
tmp = (train.groupby(['air_store_id', 'day_of_week'])
                  .apply(lambda g: calc_shifted_ewm(g['visitors'], 0.1, days=39))
                  .reset_index().set_index('level_2').sort_index()
                  .rename(columns={'visitors' : 'ewm'})
                  .drop(['air_store_id','day_of_week'], axis=1))

In [36]:
tmp = pd.concat([train, tmp], axis=1)[['air_store_id', 'visit_date', 'ewm']]

In [37]:
tmp.head()

,air_store_id,visit_date,ewm
0,air_00a91d42b08b08d9,2016-07-01,NaN
1,air_00a91d42b08b08d9,2016-07-02,NaN
2,air_00a91d42b08b08d9,2016-07-04,NaN
3,air_00a91d42b08b08d9,2016-07-05,NaN
4,air_00a91d42b08b08d9,2016-07-06,NaN


In [38]:
all_data['visit_date'] = pd.to_datetime(all_data['visit_date'])
tmp['visit_date'] = pd.to_datetime(tmp['visit_date'])

In [39]:
#all_data['visit_date'] = all_data['visit_date'].dt.date
#tmp['visit_date'] = tmp['visit_date'].dt.date

In [40]:
train = train.reset_index().drop('index',axis=1)
test = test.reset_index().drop('index',axis=1)

In [41]:
print("Train min: " + str(train['visit_date'].min()))
print("Train max:" + str(train['visit_date'].max()))
print("Test min: " + str(test['visit_date'].min()))
print("Test max:" + str(test['visit_date'].max()))
print("tmp min: " + str(tmp['visit_date'].min()))
print("tmp max:" + str(tmp['visit_date'].max()))
print("Difference: " + str(test['visit_date'].max() - train['visit_date'].max()))

Train min: 2016-01-01
Train max:2017-04-22
Test min: 2017-04-23
Test max:2017-05-31
tmp min: 2016-01-01 00:00:00
tmp max:2017-04-22 00:00:00
Difference: 39 days, 0:00:00


In [42]:
tmp['visit_date'] += dt.timedelta(days=39)

In [43]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 282487 entries, 0 to 282486
Data columns (total 73 columns):
air_store_id            282487 non-null int64
diff_10_9               282487 non-null float64
diff_11_10              282487 non-null float64
diff_12_11              282487 non-null float64
diff_8_7                282474 non-null float64
diff_9_8                282481 non-null float64
eight_weeks_ago         221561 non-null float64
eleven_weeks_ago        207193 non-null float64
nine_weeks_ago          216946 non-null float64
priorMax                249026 non-null float64
priorMean               249026 non-null float64
priorMin                249026 non-null float64
priorSum                282487 non-null float64
seven_weeks_ago         226112 non-null float64
six_weeks_ago           230884 non-null float64
ten_weeks_ago           211815 non-null float64
visit_date              282487 non-null datetime64[ns]
visitors                250468 non-null float64
vmax_6weekago_10    

In [44]:
tmp.shape

(250468, 3)

In [45]:
all_data = pd.merge(all_data, tmp, on=['air_store_id', 'visit_date'], how='left')

In [46]:
train = all_data[:train_len]
test = all_data[train_len:]

In [47]:
train = train.reset_index().drop('index',axis=1)
test = test.reset_index().drop('index',axis=1)

In [48]:
all_data['visit_date'] = pd.to_datetime(all_data['visit_date'])
#all_data['visit_date'] = all_data['visit_date'].dt.date

In [49]:
del all_data; gc.collect();

In [50]:
test.isnull().sum()[test.isnull().sum()>0]

diff_8_7                 13
diff_9_8                  6
eight_weeks_ago        4377
eleven_weeks_ago       4589
nine_weeks_ago         4439
priorMax               1632
priorMean              1632
priorMin               1632
seven_weeks_ago        4395
six_weeks_ago          4349
ten_weeks_ago          4545
visitors              32019
vmax_6weekago_7          13
vmax_6weekago_8           6
vmean_6weekago_7         13
vmean_6weekago_8          6
vmedian_6weekago_7       13
vmedian_6weekago_8        6
vmin_6weekago_7          13
vmin_6weekago_8           6
avg_temperature        3276
high_temperature       3276
low_temperature        3276
precipitation         12730
hours_sunlight         4641
ewm                   32019
dtype: int64

In [51]:
train.columns

Index(['air_store_id', 'diff_10_9', 'diff_11_10', 'diff_12_11', 'diff_8_7',
       'diff_9_8', 'eight_weeks_ago', 'eleven_weeks_ago', 'nine_weeks_ago',
       'priorMax', 'priorMean', 'priorMin', 'priorSum', 'seven_weeks_ago',
       'six_weeks_ago', 'ten_weeks_ago', 'visit_date', 'visitors',
       'vmax_6weekago_10', 'vmax_6weekago_11', 'vmax_6weekago_12',
       'vmax_6weekago_7', 'vmax_6weekago_8', 'vmax_6weekago_9',
       'vmean_6weekago_10', 'vmean_6weekago_11', 'vmean_6weekago_12',
       'vmean_6weekago_7', 'vmean_6weekago_8', 'vmean_6weekago_9',
       'vmedian_6weekago_10', 'vmedian_6weekago_11', 'vmedian_6weekago_12',
       'vmedian_6weekago_7', 'vmedian_6weekago_8', 'vmedian_6weekago_9',
       'vmin_6weekago_10', 'vmin_6weekago_11', 'vmin_6weekago_12',
       'vmin_6weekago_7', 'vmin_6weekago_8', 'vmin_6weekago_9', 'latitude',
       'longitude', 'genre_name', 'area_name', 'prefecture', 'city', 'area_id',
       'n200mt_cluster_id', 'n400mt_cluster_id', 'n1000mt_cluster_

In [52]:
test.columns

Index(['air_store_id', 'diff_10_9', 'diff_11_10', 'diff_12_11', 'diff_8_7',
       'diff_9_8', 'eight_weeks_ago', 'eleven_weeks_ago', 'nine_weeks_ago',
       'priorMax', 'priorMean', 'priorMin', 'priorSum', 'seven_weeks_ago',
       'six_weeks_ago', 'ten_weeks_ago', 'visit_date', 'visitors',
       'vmax_6weekago_10', 'vmax_6weekago_11', 'vmax_6weekago_12',
       'vmax_6weekago_7', 'vmax_6weekago_8', 'vmax_6weekago_9',
       'vmean_6weekago_10', 'vmean_6weekago_11', 'vmean_6weekago_12',
       'vmean_6weekago_7', 'vmean_6weekago_8', 'vmean_6weekago_9',
       'vmedian_6weekago_10', 'vmedian_6weekago_11', 'vmedian_6weekago_12',
       'vmedian_6weekago_7', 'vmedian_6weekago_8', 'vmedian_6weekago_9',
       'vmin_6weekago_10', 'vmin_6weekago_11', 'vmin_6weekago_12',
       'vmin_6weekago_7', 'vmin_6weekago_8', 'vmin_6weekago_9', 'latitude',
       'longitude', 'genre_name', 'area_name', 'prefecture', 'city', 'area_id',
       'n200mt_cluster_id', 'n400mt_cluster_id', 'n1000mt_cluster_

In [53]:
#train = train.fillna(0) ##0.5171185851031399
#test = test.fillna(0)
train = train.fillna(np.nan) ##0.5185937979429444
test = test.fillna(np.nan)

In [54]:
# %load prepareData.py
def prepareData(trainIn, valIn, testIn = None):
    stat1 = trainIn.groupby(["air_store_id","day_of_week"],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_store_day'})
    stat2 = trainIn.groupby(["air_store_id","day_of_week"],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_store_day'})
        
    stat3 = trainIn.groupby(["area_id","day_of_week"],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_area'})
    stat4 = trainIn.groupby(["area_id","day_of_week"],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_area'})

    stat5 = trainIn.groupby(["n200mt_cluster_id","day_of_week"],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_200mt'})
    stat6 = trainIn.groupby(["n200mt_cluster_id","day_of_week"],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_200mt'})

    stat7 = trainIn.groupby(["n400mt_cluster_id","day_of_week"],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_400mt'})
    stat8 = trainIn.groupby(["n400mt_cluster_id","day_of_week"],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_400mt'})
    
    stat9 = trainIn.groupby(["n1000mt_cluster_id","day_of_week"],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_1000mt'})
    stat10 = trainIn.groupby(["n1000mt_cluster_id","day_of_week"],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_1000mt'})
    
    stat11 = trainIn.groupby(["n200mt_cluster_id","genre_name"],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_genre_200mt'})
    stat12 = trainIn.groupby(["n200mt_cluster_id","genre_name"],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_genre_200mt'})
    
    stat13 = trainIn.groupby(["n400mt_cluster_id","genre_name"],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_genre_400mt'})
    stat14 = trainIn.groupby(["n400mt_cluster_id","genre_name"],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_genre_400mt'})
    
    stat15 = trainIn.groupby(["n1000mt_cluster_id","genre_name"],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_genre_1000mt'})
    stat16 = trainIn.groupby(["n1000mt_cluster_id","genre_name"],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_genre_1000mt'})
    
    #####################################
    stat17 = trainIn.groupby(["air_store_id","day_of_week",'week_seq_id'],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_store_day'})
    #stat18 = trainIn.groupby(["air_store_id","day_of_week",'week_seq_id'],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_store_day'})
    #stat17 = pd.merge(stat17, stat18, on=['air_store_id','day_of_week','week_seq_id'])
    #del stat18; gc.collect()
    
    stat_6wago_day = stat17.copy()
    stat_6wago_day.loc[:,'week_seq_id'] = np.nan
    stat_6wago_day.loc[:,'week_seq_id'] = stat17.loc[:,'week_seq_id'] + 6
    stat_6wago_day = stat_6wago_day.rename(columns={'visitors_sum_store_day' : 'visitors_6wks_wago'})
    
    stat_7wago_day = stat17.copy()
    stat_7wago_day.loc[:,'week_seq_id'] = np.nan
    stat_7wago_day.loc[:,'week_seq_id'] = stat17.loc[:,'week_seq_id'] + 7
    stat_7wago_day = stat_7wago_day.rename(columns={'visitors_sum_store_day' : 'visitors_7wks_wago'})
   
    stat_8wago_day = stat17.copy()
    stat_8wago_day.loc[:,'week_seq_id'] = np.nan
    stat_8wago_day.loc[:,'week_seq_id'] = stat17.loc[:,'week_seq_id'] + 8
    stat_8wago_day = stat_8wago_day.rename(columns={'visitors_sum_store_day' : 'visitors_8wks_wago'})

    stat_9wago_day = stat17.copy()
    stat_9wago_day.loc[:,'week_seq_id'] = np.nan
    stat_9wago_day.loc[:,'week_seq_id'] = stat17.loc[:,'week_seq_id'] + 9
    stat_9wago_day = stat_9wago_day.rename(columns={'visitors_sum_store_day' : 'visitors_9wks_wago'})
    
    stat_10wago_day = stat17.copy()
    stat_10wago_day.loc[:,'week_seq_id'] = np.nan
    stat_10wago_day.loc[:,'week_seq_id'] = stat17.loc[:,'week_seq_id'] + 10
    stat_10wago_day = stat_10wago_day.rename(columns={'visitors_sum_store_day' : 'visitors_10wks_wago'})
    
    #################################
    stat18 = trainIn.groupby(["air_store_id","week_seq_id"],as_index=False)['visitors'].sum().rename(columns={'visitors':'visitors_sum_store_week'})
    stat19 = trainIn.groupby(["air_store_id","week_seq_id"],as_index=False)['visitors'].mean().rename(columns={'visitors':'visitors_mean_store_week'})                  
    stat18 = pd.merge(stat18, stat19, on=['air_store_id','week_seq_id'])
    del stat19;
    
    stat_6wago = stat18.copy()
    stat_6wago.loc[:,'week_seq_id'] = np.nan
    stat_6wago.loc[:,'week_seq_id'] = stat18.loc[:,'week_seq_id'] + 6
    stat_6wago = stat_6wago.rename(columns={'visitors_sum_store_week' : 'visitors_sum_store_6wago'})
    stat_6wago = stat_6wago.rename(columns={'visitors_mean_store_week' : 'visitors_mean_store_6wago'})
    
    stat_7wago = stat18.copy()
    stat_7wago.loc[:,'week_seq_id'] = np.nan
    stat_7wago.loc[:,'week_seq_id'] = stat18.loc[:,'week_seq_id'] + 7
    stat_7wago = stat_7wago.rename(columns={'visitors_sum_store_week' : 'visitors_sum_store_7wago'})
    stat_7wago = stat_7wago.rename(columns={'visitors_mean_store_week' : 'visitors_mean_store_8wago'})
    
    stat_8wago = stat18.copy()
    stat_8wago.loc[:,'week_seq_id'] = np.nan
    stat_8wago.loc[:,'week_seq_id'] = stat18.loc[:,'week_seq_id'] + 8
    stat_8wago = stat_8wago.rename(columns={'visitors_sum_store_week' : 'visitors_sum_store_day_6wago'})
    stat_8wago = stat_8wago.rename(columns={'visitors_mean_store_week' : 'visitors_mean_store_day_6wago'}) 
    
    stat_9wago = stat18.copy()
    stat_9wago.loc[:,'week_seq_id'] = np.nan
    stat_9wago.loc[:,'week_seq_id'] = stat18.loc[:,'week_seq_id'] + 9
    stat_9wago = stat_9wago.rename(columns={'visitors_sum_store_week' : 'visitors_sum_store_9wago'})
    stat_9wago = stat_9wago.rename(columns={'visitors_mean_store_week' : 'visitors_mean_store_9wago'})
    
    stat_10wago = stat18.copy()
    stat_10wago.loc[:,'week_seq_id'] = np.nan
    stat_10wago.loc[:,'week_seq_id'] = stat18.loc[:,'week_seq_id'] + 10
    stat_10wago = stat_10wago.rename(columns={'visitors_sum_store_week' : 'visitors_sum_store_10wago'})
    stat_10wago = stat_10wago.rename(columns={'visitors_mean_store_week' : 'visitors_mean_store_10wago'}) 
    
    ##############################
    trainIn = pd.merge(trainIn, stat1, on = ["air_store_id", "day_of_week"], how='left')
    trainIn = pd.merge(trainIn, stat2, on = ["air_store_id", "day_of_week"], how='left')
    trainIn = pd.merge(trainIn, stat3, on = ["area_id", "day_of_week"], how='left')
    trainIn = pd.merge(trainIn, stat4, on = ["area_id", "day_of_week"], how='left')
    
    trainIn = pd.merge(trainIn, stat_6wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    trainIn = pd.merge(trainIn, stat_7wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    trainIn = pd.merge(trainIn, stat_8wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    trainIn = pd.merge(trainIn, stat_9wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    trainIn = pd.merge(trainIn, stat_10wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    
    trainIn = pd.merge(trainIn, stat_6wago, on = ["air_store_id", "week_seq_id"], how='left')
    trainIn = pd.merge(trainIn, stat_7wago, on = ["air_store_id", "week_seq_id"], how='left')
    trainIn = pd.merge(trainIn, stat_8wago, on = ["air_store_id", "week_seq_id"], how='left')
    trainIn = pd.merge(trainIn, stat_9wago, on = ["air_store_id", "week_seq_id"], how='left')
    trainIn = pd.merge(trainIn, stat_10wago, on = ["air_store_id", "week_seq_id"], how='left')
    
    trainIn = pd.merge(trainIn, stat5, on = ["n200mt_cluster_id", "day_of_week"], how='left')
    trainIn = pd.merge(trainIn, stat6, on = ["n200mt_cluster_id", "day_of_week"], how='left')
    trainIn = pd.merge(trainIn, stat7, on = ["n400mt_cluster_id", "day_of_week"], how='left')
    trainIn = pd.merge(trainIn, stat8, on = ["n400mt_cluster_id", "day_of_week"], how='left')
    trainIn = pd.merge(trainIn, stat9, on = ["n1000mt_cluster_id", "day_of_week"], how='left')
    trainIn = pd.merge(trainIn, stat10, on = ["n1000mt_cluster_id", "day_of_week"], how='left')
    
    trainIn = pd.merge(trainIn, stat11, on = ["n200mt_cluster_id", "genre_name"], how='left')
    trainIn = pd.merge(trainIn, stat12, on = ["n200mt_cluster_id", "genre_name"], how='left')
    trainIn = pd.merge(trainIn, stat13, on = ["n400mt_cluster_id", "genre_name"], how='left')
    trainIn = pd.merge(trainIn, stat14, on = ["n400mt_cluster_id", "genre_name"], how='left')
    trainIn = pd.merge(trainIn, stat15, on = ["n1000mt_cluster_id", "genre_name"], how='left')
    trainIn = pd.merge(trainIn, stat16, on = ["n1000mt_cluster_id", "genre_name"], how='left')
    
    ############################
    testIn = pd.merge(testIn, stat1, on = ["air_store_id", "day_of_week"], how='left')
    testIn = pd.merge(testIn, stat2, on = ["air_store_id", "day_of_week"], how='left')
    testIn = pd.merge(testIn, stat3, on = ["area_id", "day_of_week"], how='left')
    testIn = pd.merge(testIn, stat4, on = ["area_id", "day_of_week"], how='left')

    testIn = pd.merge(testIn, stat_6wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    testIn = pd.merge(testIn, stat_7wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    testIn = pd.merge(testIn, stat_8wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    testIn = pd.merge(testIn, stat_9wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    testIn = pd.merge(testIn, stat_10wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')

    testIn = pd.merge(testIn, stat_6wago, on = ["air_store_id", "week_seq_id"], how='left')
    testIn = pd.merge(testIn, stat_7wago, on = ["air_store_id", "week_seq_id"], how='left')
    testIn = pd.merge(testIn, stat_8wago, on = ["air_store_id", "week_seq_id"], how='left')
    testIn = pd.merge(testIn, stat_9wago, on = ["air_store_id", "week_seq_id"], how='left')
    testIn = pd.merge(testIn, stat_10wago, on = ["air_store_id", "week_seq_id"], how='left')

    testIn = pd.merge(testIn, stat5, on = ["n200mt_cluster_id", "day_of_week"], how='left')
    testIn = pd.merge(testIn, stat6, on = ["n200mt_cluster_id", "day_of_week"], how='left')
    testIn = pd.merge(testIn, stat7, on = ["n400mt_cluster_id", "day_of_week"], how='left')
    testIn = pd.merge(testIn, stat8, on = ["n400mt_cluster_id", "day_of_week"], how='left')
    testIn = pd.merge(testIn, stat9, on = ["n1000mt_cluster_id", "day_of_week"], how='left')
    testIn = pd.merge(testIn, stat10, on = ["n1000mt_cluster_id", "day_of_week"], how='left')

    testIn = pd.merge(testIn, stat11, on = ["n200mt_cluster_id", "genre_name"], how='left')
    testIn = pd.merge(testIn, stat12, on = ["n200mt_cluster_id", "genre_name"], how='left')
    testIn = pd.merge(testIn, stat13, on = ["n400mt_cluster_id", "genre_name"], how='left')
    testIn = pd.merge(testIn, stat14, on = ["n400mt_cluster_id", "genre_name"], how='left')
    testIn = pd.merge(testIn, stat15, on = ["n1000mt_cluster_id", "genre_name"], how='left')
    testIn = pd.merge(testIn, stat16, on = ["n1000mt_cluster_id", "genre_name"], how='left')
    
    ##################################
    valIn = pd.merge(valIn, stat1, on = ["air_store_id", "day_of_week"], how='left')
    valIn = pd.merge(valIn, stat2, on = ["air_store_id", "day_of_week"], how='left')
    valIn = pd.merge(valIn, stat3, on = ["area_id", "day_of_week"], how='left')
    valIn = pd.merge(valIn, stat4, on = ["area_id", "day_of_week"], how='left')
    
    valIn = pd.merge(valIn, stat_6wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    valIn = pd.merge(valIn, stat_7wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    valIn = pd.merge(valIn, stat_8wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    valIn = pd.merge(valIn, stat_9wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')
    valIn = pd.merge(valIn, stat_10wago_day, on = ["air_store_id", "day_of_week", "week_seq_id"], how='left')

    valIn = pd.merge(valIn, stat_6wago, on = ["air_store_id", "week_seq_id"], how='left')
    valIn = pd.merge(valIn, stat_7wago, on = ["air_store_id", "week_seq_id"], how='left')
    valIn = pd.merge(valIn, stat_8wago, on = ["air_store_id", "week_seq_id"], how='left')
    valIn = pd.merge(valIn, stat_9wago, on = ["air_store_id", "week_seq_id"], how='left')
    valIn = pd.merge(valIn, stat_10wago, on = ["air_store_id", "week_seq_id"], how='left')
    
    valIn = pd.merge(valIn, stat5, on = ["n200mt_cluster_id", "day_of_week"], how='left')
    valIn = pd.merge(valIn, stat6, on = ["n200mt_cluster_id", "day_of_week"], how='left')
    valIn = pd.merge(valIn, stat7, on = ["n400mt_cluster_id", "day_of_week"], how='left')
    valIn = pd.merge(valIn, stat8, on = ["n400mt_cluster_id", "day_of_week"], how='left')
    valIn = pd.merge(valIn, stat9, on = ["n1000mt_cluster_id", "day_of_week"], how='left')
    valIn = pd.merge(valIn, stat10, on = ["n1000mt_cluster_id", "day_of_week"], how='left')
    
    valIn = pd.merge(valIn, stat11, on = ["n200mt_cluster_id", "genre_name"], how='left')
    valIn = pd.merge(valIn, stat12, on = ["n200mt_cluster_id", "genre_name"], how='left')
    valIn = pd.merge(valIn, stat13, on = ["n400mt_cluster_id", "genre_name"], how='left')
    valIn = pd.merge(valIn, stat14, on = ["n400mt_cluster_id", "genre_name"], how='left')
    valIn = pd.merge(valIn, stat15, on = ["n1000mt_cluster_id", "genre_name"], how='left')
    valIn = pd.merge(valIn, stat16, on = ["n1000mt_cluster_id", "genre_name"], how='left')
 
    return (trainIn, valIn, testIn)

In [55]:
col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors','prefecture','city']]

In [56]:
#X = train[train['visit_date'] < dt.date(2017, 2, 21)]
#X = train[col].copy()
#y = np.log1p(train['visitors']).copy()

In [57]:
X = train.sort_values(['air_store_id', 'visit_date']).copy()

In [58]:
#test = test[col]

In [59]:
K = 5
kf = model_selection.KFold(n_splits = K, shuffle = True)

In [60]:
def RMSLE(y, pred):
    return metrics.mean_squared_error(np.log1p(y), np.log1p(pred))**0.5

In [61]:
y_test_pred = 0
y_train_pred = np.zeros(len(X))
#y_train_pred = X['visitors'].copy
y_train_pred
bestIters = []

In [62]:
y_train_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [63]:
train.shape

(250468, 74)

In [64]:
y_train_pred.shape

(250468,)

In [65]:
params = {}
params['num_leaves'] = 70,
params['metric'] = 'rmse'
params['max_depth'] = 8,
#params['min_data_in_leaf'] = 20,
#params['min_sum_hessian_in_leaf'] = 1e-3,
params['learning_rate'] = 0.02,
params['num_threads'] = -1, 
params['feature_fraction'] = 0.7

In [66]:
for i, (train_index, val_index) in enumerate(kf.split(X)):
    params['feature_fraction_seed'] = 17*i+2,
    params['bagging_fraction'] = 0.7,
    params['bagging_seed'] = 17*i+2
    
    train, val, test2 = prepareData(X.iloc[train_index, :].copy(), X.iloc[val_index, :].copy(), test.copy())
    train = train.sort_values(['air_store_id', 'visit_date'])
    val = val.sort_values(['air_store_id', 'visit_date'])
    test2 = test2.sort_values(['air_store_id', 'visit_date'])
    train = train.fillna(0)
    val = val.fillna(0)
    test2 = test.fillna(0)
    
    X_train, y_train = train[col], np.log1p(train['visitors'])
    X_valid, y_valid = val[col], np.log1p(val['visitors'])
    print("\nFold ", i)
    
    d_train = lgb.Dataset(X_train[col], y_train)
    d_valid = lgb.Dataset(X_valid[col], y_valid)
    watchlist = [d_valid]
    
    model = lgb.train(params, train_set=d_train, num_boost_round=10000, valid_sets=watchlist,
                      verbose_eval=100,early_stopping_rounds=50)
    bestIter = model.best_iteration
    bestIters.append(bestIter)
    
    val_pred = model.predict(X_valid,num_iteration =bestIter)
    val_pred[val_pred < 0] = 0
    val_pred = np.expm1(val_pred)
    val_pred[val_pred < 1] = 1
    y_train_pred[val_index] = np.array(val_pred) 
    print('RMSLE LGB Regressor, validation set, fold ', i, ': ', RMSLE(val['visitors'], val_pred))

    test_pred = model.predict(test2[col],num_iteration =bestIter)
    test_pred[test_pred < 0] = 0
    test_pred = np.expm1(test_pred)
    test_pred[test_pred < 1] = 1
    y_test_pred += test_pred

    del X_train, X_valid, y_train, y_valid, train, test2


Fold  0
Training until validation scores don't improve for 50 rounds.
[100]	valid_0's rmse: 0.532803
[200]	valid_0's rmse: 0.516248
[300]	valid_0's rmse: 0.511126
[400]	valid_0's rmse: 0.508768
[500]	valid_0's rmse: 0.507496
[600]	valid_0's rmse: 0.506409
[700]	valid_0's rmse: 0.505751
[800]	valid_0's rmse: 0.505293
[900]	valid_0's rmse: 0.504872
[1000]	valid_0's rmse: 0.504507
[1100]	valid_0's rmse: 0.504166
[1200]	valid_0's rmse: 0.503845
[1300]	valid_0's rmse: 0.503552
[1400]	valid_0's rmse: 0.503334
[1500]	valid_0's rmse: 0.503091
[1600]	valid_0's rmse: 0.502931
[1700]	valid_0's rmse: 0.502789
[1800]	valid_0's rmse: 0.502619
[1900]	valid_0's rmse: 0.502412
[2000]	valid_0's rmse: 0.502269
[2100]	valid_0's rmse: 0.502182
[2200]	valid_0's rmse: 0.502084
[2300]	valid_0's rmse: 0.50198
[2400]	valid_0's rmse: 0.501866
[2500]	valid_0's rmse: 0.501775
[2600]	valid_0's rmse: 0.501695
[2700]	valid_0's rmse: 0.501645
[2800]	valid_0's rmse: 0.501587
Early stopping, best iteration is:
[2808]	v

In [67]:
y_test_pred /= K

In [68]:
print('RMSLE LGB Regressor, full validtion, fold  ' + str(RMSLE(X['visitors'].values, y_train_pred)))

RMSLE LGB Regressor, full validtion, fold  0.502869173444738


In [74]:
submission = pd.DataFrame()
submission['id'] = str(test['air_store_id']) + "_" + str(test['visit_date'])

In [76]:
submission['visitors'] = y_test_pred

In [77]:
submission.to_csv('../Submission/submission10.csv', float_format='%.6f', index=False)